# Model Training and Validation Pipeline
Now that you have created features, you can use them to train one or more models. In this section, you will generate feature vectors with multiple features from one or more feature sets and feed them into an automated ML training and testing pipeline to create high-quality models.

The ML pipeline can be triggered and tracked manually during the interactive devel‐ opment, or it can be saved (into Git) and be executed automatically on a given schedule or as a reaction to different events (such as code modification, CI/CD, data changes, model drift, and so on). See [MLRun project and CI/CD documentation](https://docs.mlrun.org/en/stable/projects/project.html) for details.


### Saving and loading projects from GIT

After you saved your project and its elements (functions, workflows, artifacts, etc.) you can commit all your changes to a 
GIT repository. This can be done using standard GIT tools or using MLRun `project` methods such as `pull`, `push`, 
`remote`, which calls the Git API for you.

Projects can then be loaded from Git using MLRun `load_project` method, for example: 

    project = mlrun.load_project("./myproj", "git://github.com/mlrun/project-demo.git", name=project_name)
    
or using MLRun CLI:

    mlrun project -n myproj -u "git://github.com/mlrun/project-demo.git" ./myproj
    
Projects can be loaded or created by using MLRun `get_or_create_project` method.
    
Read [CI/CD integration](../../projects/ci-integration.html) for more details.

In [1]:
import mlrun
project = mlrun.get_or_create_project(
    name="fraud-demo",
    context="./",
    user_project=True,
    )

> 2025-01-28 19:19:46,103 [info] Project loaded successfully: {"project_name":"fraud-demo-jovyan"}


## Creating and Evaluating a Feature Vector

Models are trained with multiple features, which can arrive from different feature sets and be collected into training (feature) vectors. Feature stores know how to correctly combine the features into a vector by implementing smart JOINs and assessing the time dimension (time traveling).
To define a feature vector, you need to specify a name, the list of features it contains, the target features (labels), and other optional parameters. Features are specified as `<FeatureSet>.<Feature> or <FeatureSet>.*`  (all the features in a feature set). The following part demonstrates how to create and use a feature vector.


### Create a feature vector

In [2]:
# Import MLRun's Feature Store
import mlrun.feature_store as fstore

# Define the list of features to use
features = ['events.*',
            'transactions.amount_max_2h', 
            'transactions.amount_sum_2h', 
            'transactions.amount_count_2h',
            'transactions.amount_avg_2h', 
            'transactions.amount_max_12h', 
            'transactions.amount_sum_12h',
            'transactions.amount_count_12h', 
            'transactions.amount_avg_12h', 
            'transactions.amount_max_24h',
            'transactions.amount_sum_24h', 
            'transactions.amount_count_24h', 
            'transactions.amount_avg_24h',
            'transactions.es_transportation_sum_14d', 
            'transactions.es_health_sum_14d',
            'transactions.es_otherservices_sum_14d', 
            'transactions.es_food_sum_14d',
            'transactions.es_hotelservices_sum_14d', 
            'transactions.es_barsandrestaurants_sum_14d',
            'transactions.es_tech_sum_14d', 
            'transactions.es_sportsandtoys_sum_14d',
            'transactions.es_wellnessandbeauty_sum_14d', 
            'transactions.es_hyper_sum_14d',
            'transactions.es_fashion_sum_14d', 
            'transactions.es_home_sum_14d', 
            'transactions.es_travel_sum_14d', 
            'transactions.es_leisure_sum_14d',
            'transactions.gender_F',
            'transactions.gender_M',
            'transactions.step', 
            'transactions.amount', 
            'transactions.timestamp_hour',
            'transactions.timestamp_day_of_week']

In [3]:
# Define the feature vector name for future reference
fv_name = 'transactions-fraud'

# Define the feature vector using the feature store (fstore)
transactions_fv = fstore.FeatureVector(fv_name, 
                          features, 
                          label_feature="labels.label",
                          description='Predicting a fraudulent transaction')

# Save the feature vector in the feature store
transactions_fv.save()

Once you have defined the feature vector, you can use `get_offline_features()` to generate the vector dataset and return it as a dataframe or materialize it into a file (CSV or Parquet). The next part demonstrates how to retrieve a vector, materialize it, and view its results.

In [4]:
data = transactions_fv.get_offline_features()
data.to_dataframe()

> 2025-01-28 19:19:50,233 [info] Merger detected timestamp resolution incompatibility between feature set labels and others: datetime64[us] and datetime64[ms]. Converting feature set timestamp column 'timestamp' to type datetime64[us].


,event_password_change,event_details_change,event_login,amount_max_2h,amount_sum_2h,amount_count_2h,amount_avg_2h,amount_max_12h,amount_sum_12h,amount_count_12h,...,es_home_sum_14d,es_travel_sum_14d,es_leisure_sum_14d,gender_F,gender_M,step,amount,timestamp_hour,timestamp_day_of_week,label
0,0,0,1,1.83,1.83,1.0,1.830000,1.83,1.83,1.0,...,0.0,0.0,0.0,0.0,1.0,72.0,1.83,19.0,6.0,0.0
1,0,0,1,18.72,40.22,3.0,13.406667,18.72,40.22,3.0,...,0.0,0.0,0.0,0.0,1.0,66.0,18.72,19.0,6.0,0.0
2,1,0,0,25.92,64.86,3.0,21.620000,25.92,64.86,3.0,...,0.0,0.0,0.0,0.0,1.0,27.0,25.92,19.0,6.0,0.0
3,1,0,0,24.75,30.17,2.0,15.085000,24.75,30.17,2.0,...,0.0,0.0,0.0,0.0,1.0,141.0,24.75,19.0,6.0,0.0
4,1,0,0,64.18,65.17,2.0,32.585000,64.18,65.17,2.0,...,0.0,0.0,0.0,1.0,0.0,124.0,64.18,19.0,6.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1762,1,0,0,161.75,373.89,8.0,46.736250,161.75,1103.13,36.0,...,0.0,1.0,0.0,1.0,0.0,96.0,24.02,18.0,1.0,0.0
1763,0,1,0,37.74,85.49,3.0,28.496667,68.16,653.02,24.0,...,0.0,0.0,0.0,0.0,1.0,134.0,26.81,18.0,1.0,0.0
1764,0,0,1,33.10,91.11,4.0,22.777500,121.96,1001.32,32.0,...,2.0,0.0,0.0,1.0,0.0,141.0,14.95,18.0,1.0,0.0
1765,1,0,0,71.63,182.18,7.0,26.025714,71.63,1165.94,40.0,...,0.0,0.0,0.0,0.0,1.0,101.0,13.62,18.0,1.0,0.0


In [5]:
# test get online features

In [6]:
from mlrun.feature_store import get_feature_set
transactions_set = get_feature_set("fraud-demo-jovyan/transactions")
sources_list = list(transactions_set.to_dataframe().index)

In [7]:
sources = [{"source": source} for source in sources_list[:5]]
# get online features
svc = transactions_fv.get_online_feature_service()
resp = svc.get(sources)

In [8]:
import pandas as pd
pd.DataFrame(resp)

,amount_max_2h,amount_max_12h,amount_max_24h,amount_count_2h,amount_count_12h,amount_count_24h,amount_sum_2h,amount_sum_12h,amount_sum_24h,es_transportation_sum_14d,...,amount_avg_24h,event_password_change,event_details_change,event_login,gender_F,gender_M,step,amount,timestamp_hour,timestamp_day_of_week
0,71.94,89.52,267.13,6.0,28.0,67.0,193.63,975.69,2292.97,125.0,...,34.223433,0,0,1,1,0,44,71.94,19,1
1,26.81,68.16,68.20,1.0,23.0,49.0,26.81,652.19,1212.51,124.0,...,24.745102,0,1,0,0,1,134,26.81,18,1
2,31.19,67.03,77.14,2.0,28.0,63.0,52.81,580.42,1574.16,117.0,...,24.986667,0,1,0,0,1,69,31.19,18,1
3,61.70,86.29,1965.81,2.0,24.0,52.0,70.64,769.86,3953.21,92.0,...,76.023269,1,0,0,1,0,129,8.94,19,1
4,127.84,645.02,645.02,1.0,21.0,43.0,127.84,1311.15,2029.56,71.0,...,47.199070,1,0,0,0,1,0,127.84,18,1


## Building and Running an Automated Training and Validation Pipeline

MLRun allows the building of distributed ML pipelines that can handle data processing, automated feature selection, training, optimization, testing, deployments, and so on. Pipelines are composed of steps that run or deploy custom or library (from the MLRun hub) serverless functions. Pipelines can be run locally (for debugging or small-scale tasks), on a scalable Kubernetes cluster (using Kubeflow), or in a CI/CD system.

The example consists of the following pipeline steps (all using pre-defined MLRun hub functions):

1. Materialize a feature vector (using `src/get_vector`). 
2. Select the most optimal features (using `hub://feature_selection`).
3. Train the model with multiple algorithms (using `hub://auto_trainer`).
4. Evaluate the model (using `hub://auto_trainer`).
5. Deploy the model and its application to the test cluster (using `hub://v2_model_server`). The next section will explain the model and application pipeline in detail.

Each step can accept the previous steps’ results or data, and generate results, multiple visual artifacts/charts, versioned data objects, and registered models.

We have defined the workflow in [`src/new_train_workflow.py`](./src/new_train_workflow.py). 

## Running the ML pipeline

The workflow/pipeline can be executed using the MLRun SDK (`project.run()` method) or using CLI commands (mlrun project), and can run directly from the source repo (GIT). See details in [MLRun Projects and Automation documentation](https://docs.mlrun.org/en/stable/projects/project.html).

You can set arguments and destinations for the different artifacts when you run the workflow. The pipeline progress and results are shown in the notebook. Alternatively, you can check the progress, logs, artifacts, and more, in the MLRun UI or the CI/CD system. The next part demonstrates how to run the pipeline with custom arguments using the SDK.

In [9]:
run_id = project.run(
    'main',
    arguments={'vector_name':"transactions-fraud",
               'features': features,
                'label_column':"labels.label",
              }, 
    dirty=True, watch=True)

> 2025-01-28 19:19:57,940 [warning] WARNING!, You seem to have uncommitted git changes, use .push()
> 2025-01-28 19:19:57,960 [warning] It is recommended to use k8s secret (specify secret_name), specifying aws_access_key/aws_secret_key directly is unsafe.
> 2025-01-28 19:19:57,963 [warning] It is recommended to use k8s secret (specify secret_name), specifying aws_access_key/aws_secret_key directly is unsafe.
> 2025-01-28 19:19:57,965 [warning] It is recommended to use k8s secret (specify secret_name), specifying aws_access_key/aws_secret_key directly is unsafe.
> 2025-01-28 19:19:57,967 [warning] It is recommended to use k8s secret (specify secret_name), specifying aws_access_key/aws_secret_key directly is unsafe.
> 2025-01-28 19:19:57,971 [warning] It is recommended to use k8s secret (specify secret_name), specifying aws_access_key/aws_secret_key directly is unsafe.


> 2025-01-28 19:19:57,976 [info] Storing function: {"db":null,"name":"get-vector","uid":"2cce9a72576645ab84b20b0e98cd0771"}
> 2025-01-28 19:19:58,401 [info] Merger detected timestamp resolution incompatibility between feature set labels and others: datetime64[us] and datetime64[ms]. Converting feature set timestamp column 'timestamp' to type datetime64[us].
> 2025-01-28 19:19:58,569 [info] wrote target: {'status': 'ready', 'path': 's3://mlrun/projects/fraud-demo-jovyan/FeatureStore/transactions-fraud/parquet/vectors/transactions-fraud-latest.parquet', 'partitioned': True, 'kind': 'parquet', 'size': 151178, 'updated': '2025-01-28T19:19:58.569800+00:00', 'name': 'parquet'}


project,uid,iter,start,end,state,kind,name,labels,inputs,parameters,results
fraud-demo-jovyan,...98cd0771,0,Jan 28 19:19:58,NaT,completed,run,get-vector,workflow=3711f82e5fc54517a0ced56e1eb3b965kind=localowner=jovyanhost=mlrun-jupyter-69b4c585b-ngph8,,"feature_vector=transactions-fraudfeatures=['events.*', 'transactions.amount_max_2h', 'transactions.amount_sum_2h', 'transactions.amount_count_2h', 'transactions.amount_avg_2h', 'transactions.amount_max_12h', 'transactions.amount_sum_12h', 'transactions.amount_count_12h', 'transactions.amount_avg_12h', 'transactions.amount_max_24h', 'transactions.amount_sum_24h', 'transactions.amount_count_24h', 'transactions.amount_avg_24h', 'transactions.es_transportation_sum_14d', 'transactions.es_health_sum_14d', 'transactions.es_otherservices_sum_14d', 'transactions.es_food_sum_14d', 'transactions.es_hotelservices_sum_14d', 'transactions.es_barsandrestaurants_sum_14d', 'transactions.es_tech_sum_14d', 'transactions.es_sportsandtoys_sum_14d', 'transactions.es_wellnessandbeauty_sum_14d', 'transactions.es_hyper_sum_14d', 'transactions.es_fashion_sum_14d', 'transactions.es_home_sum_14d', 'transactions.es_travel_sum_14d', 'transactions.es_leisure_sum_14d', 'transactions.gender_F', 'transactions.gender_M', 'transactions.step', 'transactions.amount', 'transactions.timestamp_hour', 'transactions.timestamp_day_of_week']label_feature=labels.labeltarget={'name': 'parquet', 'kind': 'parquet'}update_stats=True",return=


> 2025-01-28 19:19:58,676 [info] Run execution finished: {"name":"get-vector","status":"completed"}
> 2025-01-28 19:19:58,677 [info] Storing function: {"db":null,"name":"feature-selection","uid":"80715a3fa0824d2d8f7f0cb53ad4f2a8"}
> 2025-01-28 19:20:02,298 [info] votes needed to be selected: 2
> 2025-01-28 19:20:02,543 [info] Merger detected timestamp resolution incompatibility between feature set labels and others: datetime64[us] and datetime64[ms]. Converting feature set timestamp column 'timestamp' to type datetime64[us].
> 2025-01-28 19:20:02,580 [info] wrote target: {'status': 'ready', 'path': 's3://mlrun/projects/fraud-demo-jovyan/FeatureStore/short/parquet/vectors/short-latest.parquet', 'partitioned': True, 'kind': 'parquet', 'size': 598152, 'updated': '2025-01-28T19:20:02.580210+00:00', 'name': 'parquet'}


/tmp/tmp55477xtl.py:316: FutureWarning:

Call to deprecated function (or staticmethod) get_offline_features. (get_offline_features() will be removed in 1.8.0, please instead use get_feature_vector('store://feature_vector_name').get_offline_features()) -- Deprecated since version 1.6.0.



...

project,uid,iter,start,end,state,kind,name,labels,inputs,parameters,results,artifact_uris
fraud-demo-jovyan,...df0290b0,0,Jan 28 19:20:08,NaT,completed,run,evaluate,workflow=3711f82e5fc54517a0ced56e1eb3b965kind=localowner=jovyanhost=mlrun-jupyter-69b4c585b-ngph8,dataset,label_columns=labelmodel=store://models/fraud-demo-jovyan/transaction_fraud_xgboost#2:latest@3711f82e5fc54517a0ced56e1eb3b965^a3a61f9ade2fde3bc21d7f4e5df994c97c8cfd65drop_columns=label,evaluation_accuracy=0.99evaluation_f1_score=0.16666666666666666evaluation_precision_score=0.4evaluation_recall_score=0.10526315789473684,evaluation-test_set=store://datasets/fraud-demo-jovyan/evaluate_evaluation-test_set#0@3711f82e5fc54517a0ced56e1eb3b965^525f0a8eda3ae4af30ce141d571653322c8ef338evaluation-confusion-matrix=store://artifacts/fraud-demo-jovyan/evaluate_evaluation-confusion-matrix#0@3711f82e5fc54517a0ced56e1eb3b965^3ee631f35a0cb4cb237308232925cc7e433a186fevaluation-roc-curves=store://artifacts/fraud-demo-jovyan/evaluate_evaluation-roc-curves#0@3711f82e5fc54517a0ced56e1eb3b965^327318265f6042df5e9d5a2fdaa54226cd978997evaluation-calibration-curve=store://artifacts/fraud-demo-jovyan/evaluate_evaluation-calibration-curve#0@3711f82e5fc54517a0ced56e1eb3b965^6594277409e9b971f79699180aa9f088f5b6cc68


> 2025-01-28 19:20:09,156 [info] Run execution finished: {"name":"evaluate","status":"completed"}
> 2025-01-28 19:20:09,186 [info] Starting remote function deploy
2025-01-28 19:20:09  (info) Deploying function
2025-01-28 19:20:09  (info) Building
2025-01-28 19:20:09  (info) Staging files and preparing base images
2025-01-28 19:20:09  (warn) Using user provided base image, runtime interpreter version is provided by the base image
2025-01-28 19:20:09  (info) Building processor image
2025-01-28 19:23:34  (info) Build complete
2025-01-28 19:23:48  (info) Function deploy complete
> 2025-01-28 19:23:50,198 [info] Model endpoint creation task completed with state succeeded
> 2025-01-28 19:23:50,199 [info] Successfully deployed function: {"external_invocation_urls":["localhost:32354"],"internal_invocation_urls":["nuclio-fraud-demo-jovyan-serving.mlrun.svc.cluster.local:8080"]}


uid,start,end,state,kind,name,parameters,results
...98cd0771,Jan 28 19:19:58,NaT,completed,,get-vector,"feature_vector=transactions-fraudfeatures=['events.*', 'transactions.amount_max_2h', 'transactions.amount_sum_2h', 'transactions.amount_count_2h', 'transactions.amount_avg_2h', 'transactions.amount_max_12h', 'transactions.amount_sum_12h', 'transactions.amount_count_12h', 'transactions.amount_avg_12h', 'transactions.amount_max_24h', 'transactions.amount_sum_24h', 'transactions.amount_count_24h', 'transactions.amount_avg_24h', 'transactions.es_transportation_sum_14d', 'transactions.es_health_sum_14d', 'transactions.es_otherservices_sum_14d', 'transactions.es_food_sum_14d', 'transactions.es_hotelservices_sum_14d', 'transactions.es_barsandrestaurants_sum_14d', 'transactions.es_tech_sum_14d', 'transactions.es_sportsandtoys_sum_14d', 'transactions.es_wellnessandbeauty_sum_14d', 'transactions.es_hyper_sum_14d', 'transactions.es_fashion_sum_14d', 'transactions.es_home_sum_14d', 'transactions.es_travel_sum_14d', 'transactions.es_leisure_sum_14d', 'transactions.gender_F', 'transactions.gender_M', 'transactions.step', 'transactions.amount', 'transactions.timestamp_hour', 'transactions.timestamp_day_of_week']label_feature=labels.labeltarget={'name': 'parquet', 'kind': 'parquet'}update_stats=True",return=
...3ad4f2a8,Jan 28 19:19:58,NaT,completed,,feature-selection,output_vector_name=shortlabel_column=labelk=25min_votes=2ignore_type_errors=True,top_features_vector=store://feature-vectors/fraud-demo-jovyan/short
...20bb3007,Jan 28 19:20:02,NaT,completed,,train,sample=-1label_column=labeltest_size=0.1,best_iteration=2accuracy=0.99f1_score=0.16666666666666666precision_score=0.4recall_score=0.10526315789473684
...df0290b0,Jan 28 19:20:08,NaT,completed,,evaluate,label_columns=labelmodel=store://models/fraud-demo-jovyan/transaction_fraud_xgboost#2:latest@3711f82e5fc54517a0ced56e1eb3b965^a3a61f9ade2fde3bc21d7f4e5df994c97c8cfd65drop_columns=label,evaluation_accuracy=0.99evaluation_f1_score=0.16666666666666666evaluation_precision_score=0.4evaluation_recall_score=0.10526315789473684


> 2025-01-28 19:23:50,282 [info] Started run workflow fraud-demo-jovyan-main with run id = '3711f82e5fc54517a0ced56e1eb3b965' by local engine


uid,start,end,state,kind,name,parameters,results
...df0290b0,Jan 28 19:20:08,NaT,completed,run,evaluate,label_columns=labelmodel=store://models/fraud-demo-jovyan/transaction_fraud_xgboost#2:latest@3711f82e5fc54517a0ced56e1eb3b965^a3a61f9ade2fde3bc21d7f4e5df994c97c8cfd65drop_columns=label,evaluation_accuracy=0.99evaluation_f1_score=0.16666666666666666evaluation_precision_score=0.4evaluation_recall_score=0.10526315789473684
...20bb3007,Jan 28 19:20:02,NaT,completed,run,train,sample=-1label_column=labeltest_size=0.1,best_iteration=2accuracy=0.99f1_score=0.16666666666666666precision_score=0.4recall_score=0.10526315789473684
...3ad4f2a8,Jan 28 19:19:58,NaT,completed,run,feature-selection,output_vector_name=shortlabel_column=labelk=25min_votes=2ignore_type_errors=True,top_features_vector=store://feature-vectors/fraud-demo-jovyan/short
...98cd0771,Jan 28 19:19:58,NaT,completed,run,get-vector,"feature_vector=transactions-fraudfeatures=['events.*', 'transactions.amount_max_2h', 'transactions.amount_sum_2h', 'transactions.amount_count_2h', 'transactions.amount_avg_2h', 'transactions.amount_max_12h', 'transactions.amount_sum_12h', 'transactions.amount_count_12h', 'transactions.amount_avg_12h', 'transactions.amount_max_24h', 'transactions.amount_sum_24h', 'transactions.amount_count_24h', 'transactions.amount_avg_24h', 'transactions.es_transportation_sum_14d', 'transactions.es_health_sum_14d', 'transactions.es_otherservices_sum_14d', 'transactions.es_food_sum_14d', 'transactions.es_hotelservices_sum_14d', 'transactions.es_barsandrestaurants_sum_14d', 'transactions.es_tech_sum_14d', 'transactions.es_sportsandtoys_sum_14d', 'transactions.es_wellnessandbeauty_sum_14d', 'transactions.es_hyper_sum_14d', 'transactions.es_fashion_sum_14d', 'transactions.es_home_sum_14d', 'transactions.es_travel_sum_14d', 'transactions.es_leisure_sum_14d', 'transactions.gender_F', 'transactions.gender_M', 'transactions.step', 'transactions.amount', 'transactions.timestamp_hour', 'transactions.timestamp_day_of_week']label_feature=labels.labeltarget={'name': 'parquet', 'kind': 'parquet'}update_stats=True",return=


## Test the model endpoint


Now that your model is deployed using the pipeline, you can invoke it as usual:

In [10]:
# Define your serving function
serving_fn = project.get_function('serving')
serving_fn.deploy()
# Choose an id for your test
sample_id = 'C1000148617'
model_inference_path = '/v2/models/fraud/infer'

# Send our sample ID for predcition
serving_fn.invoke(path=model_inference_path,
                  body={'inputs': [[sample_id]]})

> 2025-01-28 19:23:50,349 [info] Starting remote function deploy
2025-01-28 19:23:50  (info) Deploying function
2025-01-28 19:23:50  (info) Building
2025-01-28 19:23:50  (info) Staging files and preparing base images
2025-01-28 19:23:50  (warn) Using user provided base image, runtime interpreter version is provided by the base image
2025-01-28 19:23:50  (info) Building processor image
2025-01-28 19:27:15  (info) Build complete
2025-01-28 19:27:29  (info) Function deploy complete
> 2025-01-28 19:27:32,485 [info] Model endpoint creation task completed with state succeeded
> 2025-01-28 19:27:32,485 [info] Successfully deployed function: {"external_invocation_urls":["localhost:32354"],"internal_invocation_urls":["nuclio-fraud-demo-jovyan-serving.mlrun.svc.cluster.local:8080"]}
> 2025-01-28 19:27:32,513 [info] Invoking function: {"method":"POST","path":"http://nuclio-fraud-demo-jovyan-serving.mlrun.svc.cluster.local:8080/v2/models/fraud/infer"}


{'id': '8dcbd60c-bf05-4ba9-ad73-1bfa1f6357ae',
 'model_name': 'fraud',
 'outputs': [0],
 'timestamp': '2025-01-28 19:27:32.553539+00:00'}

## Done!

You've completed part 4 - the model training with the feature store.
Proceed to [Part 5](06-real-time-serving-pipeline.ipynb) to learn how to deploy real-time application pipelines.